This will be a bidirectional RNN to translate the french words into English it will be trained to translate.
It comes with another word to process the voice and convert it into the text and feeding into the model


In [1]:
#import all libraries
import pandas as pd
import numpy as np

In [2]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional,LSTM, Dropout
from keras.layers import Embedding
from keras.optimizers.legacy import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.callbacks import ModelCheckpoint

In [3]:
import sys
import tensorflow.keras
import pandas as pd
import sklearn as sk
import scipy as sp
import tensorflow as tf
import platform

print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print(f"SciPy {sp.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python 3.10.9 (main, Mar  1 2023, 12:20:14) [Clang 14.0.6 ]
Pandas 2.0.3
Scikit-Learn 1.3.0
SciPy 1.11.1
GPU is available


In [4]:
df  =pd.read_csv('./eng_-french.csv')
df.head()

,English words/sentences,French words/sentences
0,Hi.,Salut!
1,Run!,Cours !
2,Run!,Courez !
3,Who?,Qui ?
4,Wow!,Ça alors !


In [5]:
eng = df['English words/sentences']
fr = df['French words/sentences']

In [6]:
import collections

In [7]:
english_words_counter = collections.Counter([word for sentence in eng if isinstance(sentence, str) for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in fr if isinstance(sentence, str) for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in eng for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in fr for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1082098 English words.
27393 unique English words.
10 Most common words in the English dataset:
"I" "to" "you" "the" "a" "is" "Tom" "of" "in" "have"

1177832 French words.
44918 unique French words.
10 Most common words in the French dataset:
"de" "Je" "?" "pas" "que" "à" "ne" "la" "le" "Il"


In [8]:
def tokenize(x):
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

In [9]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = 55, padding = 'post')

In [10]:
def preprocess(x, y):
   
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

In [11]:
preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer = preprocess(eng, fr)

In [12]:
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Max English sentence length: 55
Max French sentence length: 55
English vocabulary size: 14531
French vocabulary size: 30660


In [13]:
def logits_to_text(logits, tokenizer):
    
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [14]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    
    learning_rate = 0.003
    
    # Build the layers
    model = Sequential()
    model.add(Embedding(french_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(Bidirectional(GRU(256, return_sequences=True)))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(english_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

In [15]:
# Reshape the input
tmp_x = pad(preproc_french_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train 
model = bd_model(
    tmp_x.shape,
    preproc_english_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

model.summary()


2023-07-20 15:13:41.569495: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-07-20 15:13:41.569531: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-07-20 15:13:41.569542: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-07-20 15:13:41.569762: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-20 15:13:41.570919: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 55, 256)           7849216   
                                                                 
 bidirectional (Bidirection  (None, 55, 512)           789504    
 al)                                                             
                                                                 
 time_distributed (TimeDist  (None, 55, 1024)          525312    
 ributed)                                                        
                                                                 
 dropout (Dropout)           (None, 55, 1024)          0         
                                                                 
 time_distributed_1 (TimeDi  (None, 55, 14532)         14895300  
 stributed)                                                      
                                                        

In [ ]:
model.fit(tmp_x, preproc_english_sentences, batch_size=64, epochs=10, validation_split=0.2)

Epoch 1/10
2196/2196 [==============================] - 2166s 986ms/step - loss: 1.0492 - accuracy: 0.9348 - val_loss: 2.2991 - val_accuracy: 0.8572
Epoch 2/10
 332/2196 [===>..........................] - ETA: 27:38 - loss: 1.0586 - accuracy: 0.9342

In [19]:
# Assuming you have your French input sentence "merci" stored in a variable called `french_input_sentence`.
french_input_sentence = "merci"

# 1. Preprocess the French input sentence.
preproc_french_input = french_tokenizer.texts_to_sequences([french_input_sentence])
preproc_french_input = pad_sequences(preproc_french_input, padding='post')

# 2. Pad the preprocessed French input to match the length of English sentences.
tmp_french_input = pad(preproc_french_input, preproc_english_sentences.shape[1])

# 3. Reshape the padded French input to match the model's input shape.
tmp_french_input = tmp_french_input.reshape((-1, preproc_english_sentences.shape[-1]))

# 4. Get the prediction using the model.
prediction = model.predict(tmp_french_input)[0]
predicted_translation = logits_to_text(prediction, english_tokenizer)

# 5. Print the prediction.
print("French Input:", french_input_sentence)
print("Prediction:", predicted_translation)


2023-07-21 09:44:46.784847: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-21 09:44:46.954896: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-21 09:44:46.966319: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 920ms/step
French Input: merci
Prediction: help you <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
